# Colab 구독 최대한 활용하기



## 더 빠른 GPU

<p>Colab 유료 요금제 중 하나를 구매한 사용자는 프리미엄 GPU를 사용할 수 있습니다. 프리미엄 액셀러레이터를 사용하려면 메뉴의 <code>런타임 &gt; 런타임 유형 변경</code>에서 노트북의 GPU 설정을 업그레이드하세요. 사용 가능 여부에 따라 프리미엄 GPU를 선택하면 V100 또는 A100 Nvidia GPU 중 하나를 사용할 수 있습니다.</p>
<p>Colab 무료 버전에서는 Nvidia T4 GPU를 사용할 수 있으며 할당량 제한 및 가용성이 적용됩니다.</p>

언제든지 다음 셀을 실행하여 할당된 GPU를 확인할 수 있습니다. 아래 코드 셀의 실행 결과가 ‘Not connected to a GPU’인 경우 메뉴의 <code>런타임 &gt; 런타임 유형 변경</code>에서 런타임을 변경하여 GPU 가속기를 사용 설정한 다음 코드 셀을 다시 실행하면 됩니다.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul  3 19:40:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

노트북에서 GPU를 사용하려면 <code>런타임 &gt; 런타임 유형 변경</code> 메뉴를 선택한 다음 하드웨어 가속기 드롭다운을 GPU로 설정하세요.

## 추가 메모리

Colab 유료 요금제 중 하나를 구매한 사용자는 고용량 메모리 VM을 사용할 수 있습니다&#40;사용 가능한 경우&#41;.
언제든지 다음 코드 셀을 실행하여 사용 가능한 메모리 용량을 확인할 수 있습니다. 아래 코드 셀의 실행 결과가 ‘Not using a high-RAM runtime’인 경우 메뉴의 <code>런타임 &gt; 런타임 유형 변경</code>에서 고용량 RAM 런타임을 사용 설정하고 런타임 구성 드롭다운에서 고용량 RAM을 선택한 다음 코드 셀을 다시 실행하면 됩니다.

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import torch

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 0)

In [ ]:
%%capture
# Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델 이름을 설정합니다.
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B-Instruct-preview does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=32,  # LoRA 알파 값을 설정합니다.
    lora_dropout=0.05,  # 드롭아웃을 지원합니다.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

In [ ]:
import json

# JSONL 파일들의 경로 리스트
file_paths = [
    '/content/GN7.jsonl',
    '/content/MX5.jsonl',
    '/content/ST1.jsonl',
    '/content/STARIA.jsonl',
    '/content/unibus.jsonl',
    ]

    # JSONL 파일을 읽어서 리스트로 변환하는 함수
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [ ]:
# 모든 JSONL 파일의 데이터를 결합
all_data = []
for file_path in file_paths:
    all_data.extend(read_jsonl(file_path))

# 데이터셋을 위한 변환
formatted_data = []
for item in all_data:
    # item이 올바르게 파싱되었는지 확인
    if not isinstance(item, dict):
        print(f"Invalid item: {item}")
        continue
    formatted_data.append({
        'instruction': item['QUESTION'],
        'output': item['ANSWER']
    })

Invalid item: [{'QUESTION': '중앙 콘솔 박스의 위치는 어디에 있나요?', 'ANSWER': '중앙 콘솔 박스는 차량 내부의 중앙에 위치해 있습니다.'}, {'QUESTION': '자동 정차 기능(Auto Hold) 버튼의 기능은 무엇인가요?', 'ANSWER': '자동 정차 기능(Auto Hold) 버튼은 차량이 정차할 때 브레이크를 자동으로 유지해주는 기능을 합니다.'}, {'QUESTION': '무선 충전 표시등/UV-C 항균 시스템 작동 버튼의 위치는 어디에 있나요?', 'ANSWER': '무선 충전 표시등/UV-C 항균 시스템 작동 버튼은 차량 내부에 있으며, USB 충전 단자와 가까운 위치에 있습니다.'}]
Invalid item: [{'QUESTION': '주행 보조 버튼의 기능을 설명하고, 해당 버튼이 설명된 페이지 번호를 적으세요.', 'ANSWER': '주행 보조 버튼은 차량의 주행을 보조하는 기능을 수행하며, 해당 버튼에 대한 설명은 7-55와 7-76 페이지에 있습니다.'}, {'QUESTION': '엔진 오일 레벨 게이지의 위치와 관련된 페이지 번호를 적으세요.', 'ANSWER': '엔진 오일 레벨 게이지는 엔진룸에 위치하며, 관련된 설명은 9-11 페이지에 있습니다.'}, {'QUESTION': '차량의 블루투스 핸즈프리 버튼의 기능과 관련된 페이지 번호를 적으세요.', 'ANSWER': '블루투스 핸즈프리 버튼은 차량에서 핸즈프리 통화를 가능하게 하는 기능을 수행하며, 관련된 설명은 5-118 페이지에 있습니다.'}]
Invalid item: [{'QUESTION': '자동 변속기 오일의 권장 용량과 사용 가능한 오일 종류는 무엇인가요?', 'ANSWER': '자동 변속기 오일의 권장 용량은 6.0 ℓ이며, 사용 가능한 오일 종류는 SK ATF SP4M-1, MICHANG ATF SP4M-1, HYUNDAI Genuine ATF SP4M-1, S-OIL ATF SP4M-1입니다.'}, {'QUESTION': '

In [ ]:
from datasets import Dataset  # Dataset 클래스를 명시적으로 임포트합니다.

# 리스트 데이터를 Hugging Face Datasets 포맷으로 변환
dataset = Dataset.from_list(formatted_data)

In [ ]:
dataset.info

DatasetInfo(description='', citation='', homepage='', license='', features={'instruction': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [ ]:
list(dataset)

[{'instruction': '하이브리드 자동차의 주요 기능은 무엇인가요?',
  'output': '하이브리드 자동차의 주요 기능은 하이브리드 에너지 흐름도, 회생제동/ECO/파워 게이지, 고전압 배터리 충전량 (SOC) 표시계, 경고등/표시등 및 클러스터 표시창 정보, 하이브리드 시스템 경고 메시지, 관성주행 안내, 대기 환경 보호 주행, 선회 성능 보조 및 긴급 회피 조향 성능 보조, e-Ride 등이 있습니다.',
  'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n하이브리드 자동차의 주요 기능은 무엇인가요?\n\n### Response:\n하이브리드 자동차의 주요 기능은 하이브리드 에너지 흐름도, 회생제동/ECO/파워 게이지, 고전압 배터리 충전량 (SOC) 표시계, 경고등/표시등 및 클러스터 표시창 정보, 하이브리드 시스템 경고 메시지, 관성주행 안내, 대기 환경 보호 주행, 선회 성능 보조 및 긴급 회피 조향 성능 보조, e-Ride 등이 있습니다.<|eot_id|>'},
 {'instruction': '가상엔진 사운드 시스템(VESS)의 역할은 무엇인가요?',
  'output': '가상엔진 사운드 시스템(VESS)은 보행자에게 차량 접근을 인식시켜 주기 위한 장치로, 저속으로 모터 주행 시 엔진 작동음이 나지 않는 경우에 가상의 전자음을 발생시킵니다.',
  'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n가상엔진 사운드 시스템(VESS)의 역할은 무엇인가요?\n\n### Response:\n가상엔진 사운드 시스템(VESS)은 보행자에게 차량 접근을 인식시켜 

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer

# 모델과 토크나이저 로드
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  torch_dtype=torch.float16,
                                                  load_in_8bit=True,
                                                  device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


# "teddylee777/QA-Dataset-mini" 데이터셋을 불러옵니다. 훈련 데이터만 사용합니다.
# dataset = load_dataset("teddylee777/QA-Dataset-mini", split="train")

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

## 모델 훈련하기

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `d

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
14.271 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 944 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,2.936900,2.844289
2,2.942000,2.795965
3,2.706200,2.610667
4,2.496600,2.281482
5,2.128500,1.975698
6,2.040100,1.721208
7,1.782600,1.592000
8,1.621600,1.538739
9,1.741000,1.491600
10,1.411400,1.437757


In [ ]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

2382.9183 seconds used for training.
39.72 minutes used for training.
Peak reserved memory = 20.615 GB.
Peak reserved memory for training = 6.344 GB.
Peak reserved memory % of max memory = 52.105 %.
Peak reserved memory for training % of max memory = 16.035 %.


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "ddualab에대해 알려주세요.",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
ddualab에대해 알려주세요.

### Response:
ddualab은 디지털 데이타 라이브러리 앱입니다. 이 앱은 1차 디지털 데이타 라이브러리에서 제공하는 데이터를 기반으로 차량의 주요 기능과 사용 방법을 설명하는 가이드를 제공합니다. ddualab은 1차 디지털 데이타 라이브러리에서 제공하는 데이터를 기반으로 차량의 주요 기능과 사용 방법을 설명하는 가이드를 제공하며, 이 가이드는 차량의 시동이 'ON' 상태일 때만 제공됩니다. 또한, ddualab은 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이 'ON' 상태일 때만 제공되며, 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이 'ON' 상태일 때만 제공되며, 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이 'ON' 상태일 때만 제공되며, 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이 'ON' 상태일 때만 제공되며, 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이 'ON' 상태일 때만 제공되며, 차량의 전원이 꺼져 있으면 시동 버튼을 누르거나, 차량의 시동이 'ON' 상태일 때 시동 버튼을 누르면 잠금 해제됩니다. ddualab은 차량의 시동이

KeyboardInterrupt: 

In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "피보나치 수열을 이어가세요.(최대 10개)",  # 지시사항
            "1, 1, 2, 3, 5, 8",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
피보나치 수열을 이어가세요.(최대 10개)

### Response:
1, 1, 2, 3, 5, 8, 10, 13, 15, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 1

In [ ]:
model.save_pretrained("Llama-3-Open-Ko-8B-teddynote")  # 모델을 로컬에 저장합니다.
# model.push_to_hub("your_name/lora_model", token = "...") # 모델을 온라인 허브에 저장합니다.

In [ ]:
base_model = "beomi/Llama-3-Open-Ko-8B"  # 병합을 수행할 베이스 모델
huggingface_token = ""  # HuggingFace 토큰
huggingface_repo = "Llama-3-Open-Ko-8B-Instruct-teddynote"  # 모델을 업로드할 repository
save_method = (
    "merged_16bit"  # "merged_4bit", "merged_4bit_forced", "merged_16bit", "lora"
)

In [ ]:
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,  # 저장 방식을 16비트 병합으로 설정
)

In [ ]:
dataset

Dataset({
    features: ['instruction', 'output', 'text'],
    num_rows: 944
})

In [ ]:
formatted_data

[{'instruction': '하이브리드 자동차의 주요 기능은 무엇인가요?',
  'output': '하이브리드 자동차의 주요 기능은 하이브리드 에너지 흐름도, 회생제동/ECO/파워 게이지, 고전압 배터리 충전량 (SOC) 표시계, 경고등/표시등 및 클러스터 표시창 정보, 하이브리드 시스템 경고 메시지, 관성주행 안내, 대기 환경 보호 주행, 선회 성능 보조 및 긴급 회피 조향 성능 보조, e-Ride 등이 있습니다.'},
 {'instruction': '가상엔진 사운드 시스템(VESS)의 역할은 무엇인가요?',
  'output': '가상엔진 사운드 시스템(VESS)은 보행자에게 차량 접근을 인식시켜 주기 위한 장치로, 저속으로 모터 주행 시 엔진 작동음이 나지 않는 경우에 가상의 전자음을 발생시킵니다.'},
 {'instruction': 'WL_ServiceLamp 경고등이 주행 중에 켜지거나 시동 후 꺼지지 않는 경우 어떻게 해야 하나요?',
  'output': 'WL_ServiceLamp 경고등이 주행 중에 켜지거나 시동 후 꺼지지 않는 경우 가까운 당사 직영 하이테크센터나 블루핸즈에서 점검 및 정비를 받아야 합니다.'},
 {'instruction': '그린 존에서 전기 모드 주행 능력이 감소하거나 엔진으로 주행할 수 있는 상황은 무엇인가요?',
  'output': '그린 존에서 전기 모드 주행 능력이 감소하거나 엔진으로 주행할 수 있는 상황은 배터리 충전 상태가 낮은 경우, 냉난방을 위해 엔진 동력이 필요한 경우, 가속/오르막 주행을 위해 엔진 동력이 필요한 경우입니다.'},
 {'instruction': '고전압 배터리 시스템을 분해하거나 조립할 경우 어떤 위험이 발생할 수 있나요?',
  'output': '고전압 배터리 시스템을 분해하거나 조립할 경우 감전으로 인하여 사망하거나 다칠 수 있으며, 차량 성능이나 내구성에 문제가 생길 수 있습니다.'},
 {'instruction': '고전압 모터 커넥터를 임의로 분리하면 어떤 위험이 있나요?

## 모델훈련하기2

In [ ]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/944 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `d

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
20.615 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 944 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,2.740500,2.615410
2,2.690500,2.560733
3,2.471000,2.299057
4,2.185700,1.986337
5,1.868300,1.684970
6,1.754300,1.397778
7,1.389900,1.289685
8,1.324600,1.249007
9,1.373100,1.211518
10,1.240400,1.178917


Step,Training Loss,Validation Loss
1,2.740500,2.615410
2,2.690500,2.560733
3,2.471000,2.299057
4,2.185700,1.986337
5,1.868300,1.684970
6,1.754300,1.397778
7,1.389900,1.289685
8,1.324600,1.249007
9,1.373100,1.211518
10,1.240400,1.178917


In [ ]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

## 데이터셋 프롬프트 변경

In [ ]:
def prompts(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{example['instruction'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{example['output'][i]}<|eot_id|>"""
        )
    return prompt_list

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다. 입력 시퀀스의 앞부분에 더 집중 할 수 있음

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=16,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=300,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=100,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="paged_adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `d

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/944 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 944 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 300
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
100,0.770800,0.597993
200,0.542800,0.342031
300,0.338100,0.263510


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "피보나치 수열을 이어가세요.(최대 10개)",  # 지시사항
            "1, 1, 2, 3, 5, 8",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
피보나치 수열을 이어가세요.(최대 10개)

### Response:
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89,?<|end_of_text|>


In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "최한선에대해 알려주세요.",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
최한선에대해 알려주세요.

### Response:
최한선은 1992년 1월 18일 서산에서 출생했으며 현재 발표를 진행중에 있습니다.<|end_of_text|>


In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "국방인공지능응용학과 알아?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
국방인공지능응용학과 알아?

### Response:
2023년 국방 무인․로봇 및 인공지능분야 방위사업청 계약학과 주관대학으로 선정되었으며, 국방혁신의 핵심분야가 AI 및 무인, 로봇 분야로 진화 및 발전함에 따른 국방 및 방산분야 미래변화를 선도적으로 주도할 인재를 양성하는 전일제 석박사 과정으로 방위산업 관련 기업 채용보장, 국비지원, 병역특례 등이 주어지는 학과입니다.<|end_of_text|>


In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "비상탈출구 루프해치를 여는 방법은 어디에 설명되있어?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
비상탈출구 루프해치를 여는 방법은 어디에 설명되있어?

### Response:
비상탈출구 루프해치를 여는 방법은 3-11 페이지에 설명되어 있습니다.<|end_of_text|>


In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "자동그리스 시스템에서 공기(AIR) 제거 방법은?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
자동그리스 시스템에서 공기(AIR) 제거 방법은?

### Response:
자동그리스 시스템에서 공기(AIR) 제거 방법은 6-68 페이지에 설명되어 있습니다.<|end_of_text|>


In [ ]:
model.save_pretrained("Llama-3-Open-Ko-8B-hyundai")  # 모델을 로컬에 저장합니다.
# model.push_to_hub("your_name/lora_model", token = "...") # 모델을 온라인 허브에 저장합니다.

In [ ]:
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,  # 저장 방식을 16비트 병합으로 설정
)

NameError: name 'save_method' is not defined